# Get lugar de ocurrencia

In [3]:
pip install unidecode

  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)
Note: you may need to restart the kernel to use updated packages.


In [38]:
import pandas as pd
import numpy as np
import re
import unidecode
import time
from geopy.geocoders import GoogleV3

Clean the location dataset

In [17]:
df = pd.read_excel("data/dataset_cleaned.xlsx")
df.columns

Index(['Codigo_Siniestro', 'Codigo_Cliente', 'Fe_Ocurrencia', 'Codigo_Poliza',
       'Cd_Provincia', 'Cd_Ciudad', 'De_Lugar_Ocurrencia',
       'Nu_Siniestro_Relacionado', 'In_Fraude', 'De_Causa_Siniestro',
       'De_Causa_Detalle', 'De_Provincia', 'De_Ciudad', 'Hora_Ocurrencia',
       'Relacion_Reclamante', 'Evento_Catastrofico', 'Tp_Documento', 'Cd_Sexo',
       'Cd_Estado_Civil', 'Rango_Edad', 'Meses_Desde_Registro',
       'In_Lista_Negra', 'Cd_Pais_Nacimiento', 'Cd_Nacionalidad',
       'In_Inactivo', 'Fe_Inactivo', 'In_Vip', 'Cd_Provincia_Nacimiento',
       'De_Act_Economica_Adicional', 'Cd_Sucursal_Envio', 'De_Sector',
       'De_Ocupacion', 'De_Actividad', 'De_Ingreso_Anual', 'De_Profesion',
       'De_Tipo_Persona', 'Tipo_Vehiculo', 'Color', 'Marca', 'Modelo', 'Año',
       'Tipo_Transmision', 'ano', 'Clase', 'Cantidad_Parque_Vehicular',
       'Tipo_Dia', 'dia_feriado'],
      dtype='object')

In [18]:
df["MOD_Lugar_Ocurrencia"] = df["De_Lugar_Ocurrencia"].str.lower()

In [19]:
def remover_acentos(x):
    """Creamos la funcion para remover los acentos"""
    if isinstance(x, str):
        return unidecode.unidecode(x)
    else:
        return x

# Aplicar la función a la columna "DE_LUGAR_OCURRENCIA"
df["MOD_Lugar_Ocurrencia"] = df["MOD_Lugar_Ocurrencia"].apply(remover_acentos)
df

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,Nu_Siniestro_Relacionado,In_Fraude,De_Causa_Siniestro,...,Marca,Modelo,Año,Tipo_Transmision,ano,Clase,Cantidad_Parque_Vehicular,Tipo_Dia,dia_feriado,MOD_Lugar_Ocurrencia
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,,0,Accidente/Choque,...,TOYOTA,CAMRY,2001.0,CE - Automatico,2015,Automovil,773019,Weekday,0,siniestro reportado via cover.
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,,0,Accidente/Choque,...,KIA,SPORTAGE,2015.0,4x4 - Automatico,2016,Jeep,386706,Weekday,0,c/ mayaguez urb. villas dumit
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,,0,Responsabilidad Civil Personas o Cosas,...,DAIHATSU,DELTA,2007.0,Sincronico,2016,Carga,405292,Weekday,0,barahora
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,,0,Robo o Hurto Accesorios/ Partes o Piezas,...,NaN,NaN,NaN,NaN,2016,Jeep,386706,Weekday,0,arroyo hondo ii
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,,0,Accidente/Choque,...,TOYOTA,PRADO,2013.0,%Sincronico,2016,Jeep,386706,Weekday,0,avenida gustavo mejia ricart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,AUTOPISTA DUARTE,,0,Rotura de Vidrios y/o Parabrisas,...,JEEP,WRANGLER,2023.0,Renegade 4x4 - Automatico,2023,Jeep,653866,Weekday,0,autopista duarte
18739,18703,181905,2023-06-03,11470,1.0,1.0,LOS MULTIS LOS ALCARRIZOS,,0,Rotura de Vidrios y/o Parabrisas,...,ISUZU,D-MAX,2015.0,Automatico,2023,Jeep,653866,Weekend,0,los multis los alcarrizos
18740,18704,182109,2023-05-22,11471,1.0,1.0,AUTOPISTA DUARTE,,0,Rotura de Vidrios y/o Parabrisas,...,HYUNDAI,TUCSON,2014.0,GL - Automatico,2023,Jeep,653866,Weekday,0,autopista duarte
18741,18705,302838,2023-06-01,11472,1.0,1.0,CALLE CAMINO CHIQUITO,,0,Accidente/Choque,...,TOYOTA,HILUX,2023.0,Cabina Sencilla Pick-Up 4x4 - Sincronico,2023,Jeep,653866,Weekday,0,calle camino chiquito


Select only relevant features

In [24]:
df2 = df[['Codigo_Siniestro','Codigo_Cliente','Fe_Ocurrencia','Codigo_Poliza',
         'Cd_Provincia','Cd_Ciudad','De_Lugar_Ocurrencia','MOD_Lugar_Ocurrencia',
         'De_Provincia','De_Ciudad']]
df2

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,MOD_Lugar_Ocurrencia,De_Provincia,De_Ciudad
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,siniestro reportado via cover.,Distrito Nacional,Santo Domingo
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,c/ mayaguez urb. villas dumit,,
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,barahora,,
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,arroyo hondo ii,Distrito Nacional,Santo Domingo
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,avenida gustavo mejia ricart,,
...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,
18739,18703,181905,2023-06-03,11470,1.0,1.0,LOS MULTIS LOS ALCARRIZOS,los multis los alcarrizos,,
18740,18704,182109,2023-05-22,11471,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,
18741,18705,302838,2023-06-01,11472,1.0,1.0,CALLE CAMINO CHIQUITO,calle camino chiquito,,


In [ ]:
def remove_special_characters(text, remove_digits=False):
    """Funcion para remover los caracteres especiales"""
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

# Asegurarse de que todos los valores en la columna sean cadenas
df2["MOD_Lugar_Ocurrencia"] = df2["MOD_Lugar_Ocurrencia"].astype(str)

# Aplicar la función a la columna "DE_LUGAR_OCURRENCIA"
df2["MOD_Lugar_Ocurrencia"] = df2["MOD_Lugar_Ocurrencia"].apply(remove_special_characters)

In [ ]:
def categorizar_lugar(lugar):
    """Creamos una columna, para identificar donde fue el sunuestro"""
    if "avenida" in lugar.lower() or "av" in lugar.lower():
        return "Avenida"
    elif "calle" in lugar.lower():
        return "Calle"
    elif "autopista" in lugar.lower() or "aut" in lugar.lower():
        return "Autopista"
    else:
        return "Otro"

df2["Categoria_Lugar"] = df2["MOD_Lugar_Ocurrencia"].apply(categorizar_lugar)
df2["Categoria_Lugar"].value_counts()

In [28]:
df2

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,MOD_Lugar_Ocurrencia,De_Provincia,De_Ciudad,Categoria_Lugar
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,siniestro reportado via cover,Distrito Nacional,Santo Domingo,Otro
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,c mayaguez urb villas dumit,,,Otro
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,barahora,,,Otro
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,arroyo hondo ii,Distrito Nacional,Santo Domingo,Otro
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,avenida gustavo mejia ricart,,,Avenida
...,...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,,Autopista
18739,18703,181905,2023-06-03,11470,1.0,1.0,LOS MULTIS LOS ALCARRIZOS,los multis los alcarrizos,,,Otro
18740,18704,182109,2023-05-22,11471,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,,Autopista
18741,18705,302838,2023-06-01,11472,1.0,1.0,CALLE CAMINO CHIQUITO,calle camino chiquito,,,Calle


In [41]:
api_key = 'hidden_key'
geolocator = GoogleV3(api_key=api_key)
latitude_list = []
longitude_list = []

# Perform geocoding for each address
for address in df2['MOD_Lugar_Ocurrencia']:
    location = geolocator.geocode(address, components={"country":"DO"}, region="DO")
    if location:
        latitude_list.append(location.latitude)
        longitude_list.append(location.longitude)
    else:
        latitude_list.append(None)
        longitude_list.append(None)

df2['latitude'] = latitude_list
df2['longitude'] = longitude_list

print(df2)

       Codigo_Siniestro  Codigo_Cliente Fe_Ocurrencia  Codigo_Poliza  \
0                     1            7668    2015-05-29              1   
1                     2           23643    2016-02-12              2   
2                     3           29730    2016-02-09              3   
3                     4           14301    2016-02-15              4   
4                     5           24144    2016-02-03              5   
...                 ...             ...           ...            ...   
18738             18702          192804    2023-04-04          11379   
18739             18703          181905    2023-06-03          11470   
18740             18704          182109    2023-05-22          11471   
18741             18705          302838    2023-06-01          11472   
18742             18706          114156    2023-05-06          11311   

       Cd_Provincia  Cd_Ciudad  \
0              29.0        1.0   
1              29.0        1.0   
2               1.0        1.0   

C:\Users\Aneur\AppData\Local\Temp\ipykernel_7412\835576196.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['latitude'] = latitude_list
C:\Users\Aneur\AppData\Local\Temp\ipykernel_7412\835576196.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['longitude'] = longitude_list


In [42]:
df2_copy=df2.copy()

In [43]:
df2_copy

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,MOD_Lugar_Ocurrencia,De_Provincia,De_Ciudad,Categoria_Lugar,latitude,longitude
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,siniestro reportado via cover,Distrito Nacional,Santo Domingo,Otro,18.735693,-70.162651
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,c mayaguez urb villas dumit,,,Otro,18.735693,-70.162651
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,barahora,,,Otro,18.212081,-71.102408
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,arroyo hondo ii,Distrito Nacional,Santo Domingo,Otro,18.486257,-69.928257
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,avenida gustavo mejia ricart,,,Avenida,18.470057,-69.945025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,,Autopista,18.490774,-69.982795
18739,18703,181905,2023-06-03,11470,1.0,1.0,LOS MULTIS LOS ALCARRIZOS,los multis los alcarrizos,,,Otro,18.524213,-70.020341
18740,18704,182109,2023-05-22,11471,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,,Autopista,18.490774,-69.982795
18741,18705,302838,2023-06-01,11472,1.0,1.0,CALLE CAMINO CHIQUITO,calle camino chiquito,,,Calle,18.493735,-69.937733


In [45]:
df2_copy[df2_copy.longitude.isnull()]

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,MOD_Lugar_Ocurrencia,De_Provincia,De_Ciudad,Categoria_Lugar,latitude,longitude
2052,2051,19158,2016-06-29,158,6.0,1.0,CASA FIRO,casa firo,,,Otro,NaN,NaN
3834,3825,92817,2017-08-02,2612,1.0,1.0,"Calle Duarte, frente a Boca Marina",calle duarte frente a boca marina,,,Calle,NaN,NaN
4983,4972,24423,2017-09-17,1555,1.0,1.0,Coun Bar,coun bar,,,Otro,NaN,NaN
5711,5698,113322,2017-12-25,3905,1.0,1.0,Calle Francisco Villa Espesa,calle francisco villa espesa,,,Calle,NaN,NaN
7933,7919,100509,2019-04-12,5170,13.0,1.0,Hotel Majestic,hotel majestic,,,Otro,NaN,NaN
8487,8473,24474,2019-04-15,5516,1.0,1.0,6 DE NOVIEMBRE,6 de noviembre,,,Otro,NaN,NaN
8901,8887,88911,2018-05-07,5746,1.0,1.0,CALLE DUARTE PROXIMO A BOCA MARINA,calle duarte proximo a boca marina,,,Calle,NaN,NaN
13283,13267,101355,2021-06-15,4828,29.0,1.0,Palacio de Justicia,palacio de justicia,,,Otro,NaN,NaN
18487,18452,199602,2023-06-09,11317,1.0,1.0,CALLE FRANCISCO VILLA ESPESA,calle francisco villa espesa,,,Calle,NaN,NaN


In [46]:
df2_copy.to_excel("data/lon_lat_data.xlsx", index=False)